#Class Definitions

In [ ]:
import iso8601

class Dossier(object):
    """
        The Dossier object is designed to contain all metadata for a single dossier.
        It can be initialized by passing the EnergyID metadata json to the constructor,
        or by calling the parse_EID_json method.
        Parsing the JSON is optional and put in a seperate method so there is room
        to add other metadata formats.
    """
    def __init__(self, EID_json=None):
        """
            Object constructor. If the EnergyID JSON is passed, all metadata and
            datapoints are parsed and added. It is left optional though so other methods
            of initializing can be added in the future.
            
            Parameters
            ----------
            EID_json: JSON string from EnergyID containing all metadata for a single dossier
        """
        self.points = []
        if EID_json is not None:
            self._parse_EID_json(EID_json)
    
    def _parse_EID_json(self,mjson):
        """
            Method that parses the EnergyID JSON
            
            Parameters
            ----------
            mjson: JSON string from EnergyID containing all metadata for a single dossier
        """
        
        self.id = mjson['id']
        self.version = mjson['version']
        self.created = iso8601.parse_date(mjson['created']) #parse to datetime
        self.active = mjson['status'] == 'active' #parse to bool
        
        #We save the context and weatherstation metadata as raw JSON for now.
        self._context = mjson['context']
        self.weatherStations = mjson['weatherStations']
        
        #Data Points are objects. We create a list of objects here.
        for pjson in mjson['points']:
            self.points.append(Point(dossier = self, EID_json = pjson))
            
    def get_property(self,tag):
        """
            Method that parses context metadata
            
            Parameters
            ----------
            tag: String, tag of the property to be parsed
            
            Returns
            -------
            Parsed value for the given tag.
            None if the tag was not found.
        """
        try:
            res = self._context[tag]
        except KeyError:
            res = None
            
        return res

In [ ]:
class Point(object):
    """
        The Point object countains all metadata for a Data Point,
        and all coupled metrics.
    """
    def __init__(self, dossier, EID_json=None):
        """
            Construction method. EnergyID JSON is optional so that
            a data point can be constructed by via another method in the future.
            
            Parameters
            ----------
            dossier: Dossier object, parent to this Data Point
            EID_json: EnergyID JSON for this data Point (optional)
        """
        self.dossier = dossier
        
        if EID_json is not None:
            self._parse_EID_json(EID_json)
            
    def _parse_EID_json(self,mjson):
        """
            Parse the EnergyID JSON into metadata, metrics and dimensions
            
            Parameters
            ----------
            mjson: JSON String, EnergyID JSON for a single point
        """
        self.id = mjson['id']
        self.active = mjson['status'] == 'active' #parse to bool
        
        #save al other properties as raw json and parse when requested
        self._raw_json = mjson
        
        #create list of coupled metrics and dimensions
        self.metrics = [Metric(point = self, EID_json = cjson) for cjson in mjson['metrics']]
        
    def get_propery(self,tag):
        """
            Method that parses context metadata
            
            Parameters
            ----------
            tag: String, tag of the property to be parsed
            
            Returns
            -------
            Parsed value for the given tag.
            None if the tag was not found.
        """
        try:
            res = self._raw_json[tag]
        except KeyError:
            res = None
            
        return res

In [ ]:
class Metric(object):
    """
        Object contains info for a metric
    """
    def __init__(self, point, EID_json = None):
        """
            Constructor
            
            Parameters
            ----------
            point: parent Point
            EID_json: string
            
        """
        self.point = point
        
        if EID_json is not None:
            self._parse_EID_json(EID_json)
            
    def _parse_EID_json(self, mjson):
        
        self.name = mjson['name']
        self.coefficient = mjson['coefficient']
        self.dimensions = [Dimension(metric = self, EID_json = cjson) for cjson in mjson['dimensions']]

In [ ]:
class Dimension(object):
    """
        Object contains info for a dimension
    """
    def __init__(self, metric, EID_json = None):
        """
            Constructor
            
            Parameters
            ----------
            metric: parent
            EID_json: string
        """
        
        self.metric = metric
        
        if EID_json is not None:
            self._parse_EID_json(EID_json)
            
    def _parse_EID_json(self, mjson):
        self.name = mjson['name']
        self.value = mjson['value']

#Parsing

In [ ]:
import json

In [ ]:
with open('metadata_eid.json') as data_file:    
    mjson = json.load(data_file)

In [ ]:
d = Dossier(EID_json=mjson)

In [ ]:
print d.id, d.version, d.created, d.active

In [ ]:
d.get_property('category'), d.get_property('dwellingType')

In [ ]:
for w in d.weatherStations:
    print w

In [ ]:
for p in d.points:
    print p.id
    print p.get_propery('displayName')
    print p.metrics
    print '\n'

In [ ]:
for p in d.points:
    for m in p.metrics:
        print m.name, m.coefficient
        for dim in m.dimensions:
            print "\t",
            print dim.name, dim.value
    print '\n'